# Intro to Prompt Engineering with LLMs

[This course](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) is produced by the ubiquitous Andrew Ng & OpenAI Engineer Isa Fulford.

The course starts with a distinction in the functionality between 2 types of LLM:

**Base LLM**: Goos at predicting the next predicted word, so can exhibit autocomplete
behaviour. 

**Instruction Tuned LLM**: A base LLM trained to respond to instructions such as 
questions. This often involves **RLHF** - reinforcement learning with human feedback. 

The LLM output should be:

**H**elpful, **H**onest & **H**armless

## Prompt Engineering

To follow along, you'll need:
* to pip install `openai` - this is in the requirements.txt.
* to generate an api key from the [openai website](https://platform.openai.com/account/api-keys).


In [7]:
import toml
import os
from pyprojroot import here
import openai
from redlines import Redlines
from IPython.display import display, Markdown


In [ ]:
CONFIG = toml.load(os.path.join(here(), ".ignore_me", "secrets.toml"))
PROMPT_ENGINEER = CONFIG["openai"]["PROMPT_ENGINEER"]
openai.api_key = PROMPT_ENGINEER


In [ ]:
def get_answer(usr_input, mod="gpt-3.5-turbo"):
    """Get an answer from chat GPT.

    Args:
        usr_input (str): A prompt generated by the user.
        mod (str, optional): The model of GPT to use. Defaults to "gpt-3.5-turbo".

    Returns:
        str: The model response text.
    """
    mod_input = [{"role": "user", "content": usr_input}]
    resp = openai.ChatCompletion.create(
        model=mod,
        messages=mod_input,
        temperature=0 # degree of randomness 
    )
    return resp.choices[0].message["content"]

The above is not working as rate limits asssociated with my tier of key.

## Guidance on Prompts

### Use delimeters

Longer, more specific prompts are specified. You can use delimeters such as:

* `"""`
* ` ``` `
* `---`
* `<>`
* `<tag></tag>` (XML-flavoured tags)

Then use the available tags to stucture your prompt as follows:


In [ ]:
war_n_peace  = """
Some scintillating text tht you wint to summarise  / analyse / do something with \
enburing enouhg context is provided.
"""
# now for the instruction
usr_input = f"""Correct any typos in the text delimited by triple backticks\
    ```{war_n_peace}````
"""
# get_answer(usr_input)

**Response generated**:
'Some scintillating text that you want to summarise / analyse / do something with ensuring enough context is provided.'

This is also a good defence against prompt injection. By exposing the `war_n_peace` input
only to the user, chatGPT will be summarising these dynamic inputs rather than following
any instructions contained within.

### Specify structured output

You can ask for HTML, XML, JSON etc, provide enough info for the model to cope though:


In [ ]:
prompt = """
Make up 3 new science fiction movies, including their director & tagline. Return
format should be JSON with the following keys: movie_name, director_name, tagline.
"""
# get_answer(prompt)

**Response generated**:

'{\n  "movie1": {\n    "movie_name": "Galactic Odyssey",\n    "director_name": "Christopher Nolan",\n    "tagline": "Journey beyond the stars"\n  },\n  "movie2": {\n    "movie_name": "Chrono Shift",\n    "director_name": "Denis Villeneuve",\n    "tagline": "Time is not what it seems"\n  },\n  "movie3": {\n    "movie_name": "Quantum Paradox",\n    "director_name": "James Cameron",\n    "tagline": "The laws of physics are about to be broken"\n  }\n}'

### Check conditions

You can ask the model to check some assumptions prior to making a response. For example:

In [ ]:
txt = "soundgarden, queens of the stone age, mark lanegan band"

prompt = f""" 
Check whether some well-known rock band names are provided within the \
    backticked-delimiited text:
    ```{txt}```
    If band names are found, generate a new album name for each band found.
    If no band was found, return \"No known bands found.\" 
"""
# get_answer(prompt)

**Response generated**:

'New album names:\n- Soundgarden: "Echoes of the Black Hole"\n- Queens of the Stone Age: "Villains of the Valley"\n- Mark Lanegan Band: "Phantom Radio Sessions"\n\nNote: As an AI language model, I cannot determine whether these album names already exist or not.'

In [ ]:
txt = "eggs, ham"
#  get_answer(prompt)
# response: 'No known bands found.'

### Few-shot prompting

This is when you provide the model with an exemplar answer. Let's take a look:

In [ ]:
prompt = """
I need you to answer in the style of Yoda:

<Luke>: Why can't I lift these stones with the force?
<Yoda>: Work hard you must. Easy, it is not.
<Luke>: I'm hungry. Can you fix me up a womp rat stew?
"""

# get_answer(prompt)

**Response generated**:  
'`<Yoda>`: Hungry, are you? Womp rat stew, I can make. But first, patience you must have. Train your mind and body, you must, to become a Jedi.'

## Providing Think Time

Asking the model to take longer time can ensure it provides the correct answer rather
than quickly returning an incorrect answer. Also, breaking down complex tasks into 
separate instructions:

In [ ]:
txt = """"
Der Strauß, den ich gepflücket,
Grüße dich vieltausendmal!
Ich hab mich oft gebücket,
Ach, wohl eintausendmal,
Und ihn ans Herz gedrücket
Wie hunderttausendmal!.
"""
prompt = f"""
Perform the following instructions, seperate your answers with carriage returns:
1. Translate the backtick-delimited text into American English: ```{txt}```
2. Translate the same backtick-delimited text into United Kingdom English.
3. Underline the text generated in steps 2 and 3 wherever they differ.
4. Summarise the American English version of the text to be as succinct as possible.

"""
# get_answer(prompt)

**Response generated:**

'1. "The bouquet that I picked, greets you a thousand times! I have often bent down, oh, probably a thousand times, and pressed it to my heart like a hundred thousand times!"\n2. "The bouquet that I picked, greets you a thousand times! I have often bent down, oh, probably a thousand times, and pressed it to my heart like a hundred thousand times!"\n3. No differences found.\n4. The speaker greets the recipient with a bouquet that they have picked and expresses how many times they have held it close to their heart.'

## Ensuring checks are completed with accuracy

Unless you are explicit, it seems the model may skim read and come to the incorrect
conclusions. You can be explicit in your instructions to force the model to generate the
correct result.  

In [ ]:
txt = """The frequence of a wave is the velocity of the wave divided by its wavelength \
. The wavelength of red light is 700 x 10<sup>-9</sup> nm. \
The velocity of light in a vacuum is  3 x 10<sup>8</sup> m/s. \
Calculate the frequency of red light in a vacuum.

<student>:
700 x 10<sup>9</sup> / 3 x 10<sup>8</sup>
= 2.333333333333333e<sup>-1</sup> Hz
"""


response = get_completion(prompt)
print(response)

prompt = f"""Check whether the student's answer in the following backtick-delimited text
is correct: ```{txt}```
"""
# get_answer(prompt)

Model response: Yes, the student's answer is correct.

This is incorrect. The student confused the numerator & denominator, forgot the sign in
the wavelength power term, got confised with nm to m. To have the model check
thoroughly, use a more detailed prompt, such as: 


In [ ]:
prompt = f"""Check whether the student's answer in the following backtick-delimited text
is correct: ```{txt}```.
Use the following format:
- State the question in equation format
- Respond with the student's answer
- Respond with your calculated answer. 
- Compare your answer to the student's answer & determine if they match.
- Respond with an outcome and some feedback for the student.

"""
# get_answer(prompt)

**Generated response**:

Question: What is the frequency of red light in a vacuum?

Student's answer: 700 x 10<sup>9</sup> / 3 x 10<sup>8</sup> = 2.333333333333333e<sup>-1</sup> Hz

Calculated answer: c = λν, where c = 3 x 10<sup>8</sup> m/s and λ = 700 x 10<sup>-9</sup> m. Therefore, ν = c/λ = (3 x 10<sup>8</sup> m/s)/(700 x 10<sup>-9</sup> m) = 4.285714285714286 x 10<sup>14</sup> Hz.

Outcome: The student's answer is incorrect.

Feedback: The student made an error in their calculation. They divided the wavelength by the velocity instead of dividing the velocity by the wavelength. They also used nanometers instead of meters for the wavelength. The correct answer is 4.29 x 10<sup>14</sup> Hz. The student should review the formula for calculating frequency and ensure that they are using the correct units.

## Limitations

The LLM has been trained on a very large corpus, but will not perfectly recall all
details. This can result in imaginery responses, known as 'hallucinations'. To try to
limit this, have the LLM respond with the relevant text prior to summarising, to trace
back the origin of such hallucinations.

In [ ]:
prompt = """Provide a product description of the following
product:
Marks & Spencers spam niblets

"""
# get_answer(prompt)

'Unfortunately, as an AI language model, I cannot provide a product description of Marks & Spencers spam niblets as it is not a real product. It is possible that you may have misspelled the product name or it may not exist in the market. Please provide the correct product name or more information about the product so I can assist you better.'

I guess the model is too clever for my prompts...

The rest of the course is pretty straight forward in terms of describing how to iterate on prompts & summarising strengths of the chatgpt LLM. I will summarise them here and also add in some interesting code snippets introduced within the course. 


## Summarising

The example uses a product review such as those found on ebay. Asks the model to summarise the review within a certain word limit. This is then done in series for several reviews.

## Inferring

In this chapter, the product review is used to prompt the model for sentiment, topics, item name and brand etc.

## Transforming

This chapter focussed on translating to different languages. 

## Expanding 

This section states that given enough context in the prompt, the LLM are strong in generating stories, poems etc. The code asked the LLM to respond to customer complaints with automated apologies. It also pointed out that each prompt session with the LLM is discrete and context will not be remembered by the LLM between prompts. Therefore it's important to extract perinent details to pass back to the next successive prompt.

## Chatbot

This section introduces system role messages. These are intended to provide context to the behaviour of the LLM. This is to place limits on its behaviour. For example:


In [ ]:
messages =  [  
{'role':'system', 'content':'You are a customer service assistant chatbot.'},
{'role':'user', 'content':'Hello, I\'m Miles'},
{'role':'assistant', 'content': "Hi Miles! It's great to meet your acquiantance. \
Can I help you?"},
{'role':'user', 'content':'Yes please, tell me my name?'}  ]
#response = get_completion_from_messages(messages, temperature=1)
#print(response)

This won't work as the prompt sessions are seperate.

The chapter then used the `panel` package to present a text input to the user and present the chat within a UI.

ipython display was used to render HTML returned by the LLM. 

And an interesting packagew as used to show the diff between 2 related pieces of text. 

In [10]:
jumps = "The quick brown fox jumps over the lazy dog."
walks = "The quick brown fox walks past the lazy dog."


diff = Redlines(jumps,walks)
display(Markdown(diff.output_markdown))

The quick brown fox <span style="color:red;font-weight:700;text-decoration:line-through;">jumps over </span><span style="color:red;font-weight:700;">walks past </span>the lazy dog.